In [24]:
import pandas as pd
from simpledbf import Dbf5
# from sqlalchemy import create_engine
import pymysql

In [25]:
# Membaca file DBF
dbf = Dbf5('BATAS_DESA_DESEMBER_2019_DUKCAPIL_JAWA_TIMUR.dbf')
df = dbf.to_dataframe()

# Membaca file csv
df2 = pd.read_csv("pembagian_wilayah_sby.csv")

In [26]:
df = df[['DESA','PROVINSI','KAB_KOTA','KECAMATAN']]
df = df[df['KAB_KOTA']=='KOTA SURABAYA']
df['KAB_KOTA'] = df['KAB_KOTA'].str.replace('KOTA ', '')
df = df.reset_index(drop=True)
df.to_csv('CEK.csv', index=False)

In [27]:
merged_data = pd.merge(df, df2, on='KECAMATAN', how='left')

# Mengatur kolom yang ingin ditampilkan
merged_data = merged_data[['DESA', 'PROVINSI', 'KAB_KOTA', 'KECAMATAN', 'WILAYAH']]

# Menyimpan hasil gabungan ke file CSV baru
merged_data.to_csv('fix_data.csv', index=False)

In [28]:
# # Load ke db
# engine = create_engine('mysql+pymysql://root:@localhost:3306/db_bpbd')

# # Tulis DataFrame ke dalam tabel SQL
# merged_data.to_sql('wilayah_2022', con=engine, if_exists='replace', index=False)

In [29]:
def write_dataframe_to_mysql(df, table_name, conn):
    cursor = conn.cursor()
    
    # Drop tabel jika sudah ada
    drop_table_query = f"DROP TABLE IF EXISTS {table_name}"
    cursor.execute(drop_table_query)
    
    # Buat string untuk perintah SQL untuk membuat tabel baru
    columns = ", ".join([f"`{col}` TEXT" for col in df.columns])
    create_table_query = f"CREATE TABLE {table_name} ({columns})"
    cursor.execute(create_table_query)
    
    # Buat string untuk perintah SQL untuk memasukkan data
    insert_query = f"INSERT INTO {table_name} VALUES ({', '.join(['%s'] * len(df.columns))})"
    
    # Insert data
    for i, row in df.iterrows():
        cursor.execute(insert_query, tuple(row))
    
    conn.commit()
    cursor.close()


In [30]:
# Buat koneksi ke database MySQL
connection = pymysql.connect(
    host='localhost',
    user='root',
    password='',
    db='db_bpbd'
)

# Tulis DataFrame ke dalam tabel SQL
write_dataframe_to_mysql(merged_data, 'wilayah_2022', connection)

# Tutup koneksi
connection.close()
